# データの前処理

## import

In [54]:
import numpy as np
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import tensorflow.keras.layers as layers
# tensorflowのversion
print("tensorflow version is {}".format(tf.__version__))

tensorflow version is 2.3.0


## データセットの生成

In [57]:
from data_generator import DataGenerator
inputfilename = "./input/questionnaire_latest.csv"
dg = DataGenerator(inputfilename)
dg.generate()
# データセットへのアクセス
#dg.df_static_info_binary   # 静的情報（バイナリベクトル）
#dg.df_static_info_weight   # 静的情報（重み付きベクトル）
#dg.df_dynamic_info         # 動的情報
#dg.df_human_info           # 性格情報

ValueError: 'アウトドア（マリンスポーツ・釣り・BBQ・登山・ハイキングなど）, エンタメ（ボウリング場・カラオケ・ゲームセンターなど）, 観光（観光名所巡り・日帰り温泉・食べ歩きなど）, 友達との飲み会（外飲み・飲み歩きなど）, 外での買い物（アウトレットなど）' is not in list

In [35]:
inputfilename = "./input/questionnaire.csv"
data = pd.read_csv(inputfilename).drop(
    columns=['あなたの年代は？','何時から遊び始めたいですか？','何時に遊び終わりたいですか？','タイムスタンプ','所要時間(h)'])
replace_dict = {'男性':0, '女性':1, 'ない':0, 'ある':1}
data = data.replace(replace_dict)

In [36]:
train, test = train_test_split(data, test_size=0.20)
train = train.reset_index(drop=True)
test  = test.reset_index(drop=True)

In [37]:
x_train = train.drop(columns="上記の条件の際に，どの遊びをレコメンドして欲しいですか？(1つのみ)")
y_train = train.loc[:,'上記の条件の際に，どの遊びをレコメンドして欲しいですか？(1つのみ)']

x_test = test.drop(columns="上記の条件の際に，どの遊びをレコメンドして欲しいですか？(1つのみ)")
y_test = test.loc[:,'上記の条件の際に，どの遊びをレコメンドして欲しいですか？(1つのみ)']

In [38]:
x_train.shape

(60, 14)

In [39]:
y_train.shape

(60,)

In [40]:
x_test.shape

(15, 14)

In [41]:
y_test.shape

(15,)

# モデル実装

## 主成分分析

In [42]:
pca = PCA(n_components=6)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
sum(pca.explained_variance_ratio_)

0.8069409700061265

In [43]:
pca.fit(x_test)
x_test_pca = pca.transform(x_test)
sum(pca.explained_variance_ratio_)

0.9337145221758625

## MLP

In [49]:
def create_model():
    inputs = layers.Input((14,))
    x = layers.Dense(8, activation="relu")(inputs)
    x = layers.Dense(14, activation="sigmoid")(x)
    return tf.keras.models.Model(inputs=inputs, outputs=x)

def main():
    
    model = create_model()
    loss = tf.keras.losses.MeanSquaredError()
    acc = tf.keras.losses.MeanSquaredError()
    optim = tf.keras.optimizers.Adam()

    # train
    model.compile(optimizer=optim, loss=loss, metrics=[acc])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=10)

    # eval
    val_loss, val_acc = model.evaluate(x_test, y_test, batch_size=10)
    print(val_loss, val_acc)

if __name__ == "__main__":
    main()

Epoch 1/100


UnimplementedError:  Cast string to float is not supported
	 [[node Cast_1 (defined at <ipython-input-49-fb0bf5ed7362>:16) ]] [Op:__inference_train_function_2268]

Function call stack:
train_function


# 実装(不採択)

## ランダムフォレスト

In [7]:
clf = RandomForestClassifier(max_depth=5, random_state=0)

In [8]:
# original
clf.fit(x_train,y_train)

# pca
#clf.fit(x_train_pca,y_train)

RandomForestClassifier(max_depth=5, random_state=0)

In [9]:
print(clf.feature_importances_)

[0.03545779 0.03583479 0.07081688 0.07663516 0.07443875 0.07018595
 0.1078693  0.0668443  0.0823463  0.05600769 0.08187283 0.07853895
 0.08160876 0.08154253]


In [10]:
# original
clf.score(x_test,y_test)

# pca
#clf.score(x_test_pca,y_test)

0.13333333333333333